## Importing 'DDW-C18-0000.XLSX'(Language Data)  and 'DDW-0000C-13A.xlsx'(Census Data according to age) file using pandas

In [1]:
import pandas as pd
data = pd.read_excel('INPUT/DDW-C18-0000.XLSX')
census_data = pd.read_excel('INPUT/DDW-0000C-13A.xlsx')

### Creating the list of age groups included in the language data to get the appropriate data from the census data according to age
### We modify some elements of the list for easy functioning in the later part

In [2]:
age_grp = data['Unnamed: 4'][6:].unique()
age_grp = list(age_grp)
age_grp = age_grp[:9]
age_grp
age_grp[7] = '70-100+'
age_grp[8] = 'Age not stated-'
age_grp

['5-9',
 '10-14',
 '15-19',
 '20-24',
 '25-29',
 '30-49',
 '50-69',
 '70-100+',
 'Age not stated-']

### Getting the list of age column from the census data according to age, and finding the unique values from this list to get one single list of all ages as strings(for matching with above found age groups list) occuring in the complete census data for all states, UTs and India. This will help us to get the ages of age groups found above

In [3]:
age_list = census_data['C-13 APPENDIX - SINGLE YEAR AGE RETURNS BY RESIDENCE, SEX  AND LITERACY STATUS'][6:].unique()
age_list = list(age_list)
for i in range(len(age_list)):
    age_list[i] = str(age_list[i])

### Getting the indices of the starting age of an age group in the above found list so that by manipulating certain parameters we can reach the population for an age with the help of these indices. Also, adding an extra index for getting the data for the last group too

In [4]:
age_grp_ind = []
for i in range(len(age_grp)):
    age_grp_ind.append(age_list.index(age_grp[i].split("-")[0]))
age_grp_ind
age_grp_ind.append(age_grp_ind[-1]+1)
age_grp_ind

[6, 11, 16, 21, 26, 31, 51, 71, 102, 103]

### Getting the unique list of states/UTs from the census data to check whether they are similar to the ones obtained from the language data

In [5]:
name_lst = list(census_data['Unnamed: 3'][6:].unique())
for i in range(len(name_lst)):
    if name_lst[i] == "India":
        name_lst[i] = "INDIA"
    else:
        name_lst[i] = name_lst[i].split("- ")[1]
        name_lst[i] = name_lst[i].split(" (")[0]

### Checking if both have the same states with same sequence

In [6]:
state_lang = data['Unnamed: 2'][6:].unique()
yes = 0
for i in range(len(state_lang)):
    if name_lst[i] == state_lang[i]:
        yes = yes+1
print(yes)

36


### Getting the total population for each state/UT for each age group and storing that in form of a list of lists. Here, the total population is divided in literate and illiterate groups. So, We just sum them up together

In [7]:
age_pop = [[0]*len(age_grp)]*len(name_lst)
for i in range(len(name_lst)):
    temp = [0]*len(age_grp)
    for j in range(len(age_grp)):
        for k in range(age_grp_ind[j],age_grp_ind[j+1]):
            temp[j] =  temp[j] + census_data['Unnamed: 6'][((2*(i + 1))-2)*103 + k + 6] + census_data['Unnamed: 6'][((2*(i+1))-1)*103 + k + 6]
    age_pop[i] = temp

In [8]:
age_pop

[[126928126,
  132709212,
  120526449,
  111424222,
  101413965,
  308492074,
  152333999,
  39730350,
  4489802],
 [1411973,
  1413853,
  1237462,
  1160913,
  1086122,
  3069723,
  1335288,
  395965,
  15119],
 [591177, 639224, 640461, 643866, 589160, 1905011, 987526, 313030, 10163],
 [2368019,
  2583402,
  2817683,
  2776636,
  2463861,
  7481850,
  3927197,
  1149639,
  41522],
 [91194, 93464, 104418, 121244, 110237, 303748, 123507, 25438, 346],
 [1058801, 1145343, 1124110, 970068, 810184, 2419886, 1263434, 352994, 16608],
 [2498757,
  2668678,
  2677390,
  2567486,
  2231959,
  6403218,
  3064954,
  844967,
  31534],
 [1535972,
  1648147,
  1667375,
  1764060,
  1668326,
  4762353,
  1940832,
  402974,
  16702],
 [8042738,
  8380518,
  7314017,
  6426494,
  5348497,
  15938006,
  7444772,
  2081623,
  269602],
 [25061717,
  25869881,
  23040380,
  17578648,
  14579829,
  44192083,
  21596329,
  5894744,
  1622062],
 [15036277,
  13919882,
  9472695,
  8064735,
  7671705,
  2294906

### Getting the percentage of population speaking third language for all state/UTs and India

In [9]:
age_percent = [[0]*len(age_grp)]*len(name_lst)
for i in range(len(name_lst)):
    temp = [0]* len(age_grp)
    for j in range(len(age_grp)):
        temp[j] = (data['Unnamed: 8'][(list(data['Unnamed: 2']).index(name_lst[i]))+j+1])*100 / age_pop[i][j]
    age_percent[i] = temp

In [10]:
age_percent

[[1.4528757794785374,
  5.466338689434762,
  10.47630383601528,
  11.51844165445463,
  10.705794808436885,
  8.903594716018539,
  7.166069998595652,
  4.91951115457075,
  5.682455484673935],
 [5.211785211190299,
  15.743220829888255,
  27.407063812868596,
  29.580166644701197,
  25.03853158300817,
  19.798040409509262,
  15.150739016601662,
  8.563888222443904,
  10.47026919769826],
 [1.1071134364158282,
  4.612311177302479,
  8.009543125967078,
  8.206521232678849,
  7.733892321270962,
  5.9210681723097665,
  4.150473000204552,
  2.3627128390250136,
  3.6996949719570993],
 [7.716154304505158,
  28.538841419182923,
  45.763948606000035,
  45.64296508436828,
  40.69044479376069,
  31.047334549610056,
  22.44308090477763,
  11.922612228708315,
  22.099128173016712],
 [9.37123056341426,
  29.34605837541727,
  39.47882548985807,
  37.958167002078454,
  33.681976106026106,
  32.233298655464395,
  42.7749034467682,
  42.56230835757528,
  18.786127167630056],
 [0.39809180384227066,
  1.388579

### Getting the maximum percentage for each state/UT with the index for the age group which has this highest percentage

In [11]:
age_percentmax = [0] * len(name_lst)
age_percentmax_agegrp = [0] * len(name_lst)
for i in range(len(name_lst)):
    age_percentmax[i] = max(age_percent[i])
    age_percentmax_agegrp[i] = age_grp[age_percent[i].index(age_percentmax[i])]

### Changing back the age groups back to normal

In [12]:
for i in range(len(age_percentmax_agegrp)):
    if age_percentmax_agegrp[i] == '70-100+':
        age_percentmax_agegrp[i] = '70+'
    if age_percentmax_agegrp[i] == 'Age not stated-':
        age_percentmax_agegrp[i] = 'Age not stated'

### Getting the codes of the states

In [13]:
state_code = [0] * len(name_lst)
for i in range(len(name_lst)):
    state_code[i] = data['C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX'][list(data['Unnamed: 2']).index(name_lst[i])]

### Exporting the state codes, state names into a dataframe

In [14]:
data_final = {'state-code': state_code, 'state/ut': name_lst,'age-group': age_percentmax_agegrp,'percentage': age_percentmax}
age_india = pd.DataFrame(data_final)
age_india

,state-code,state/ut,age-group,percentage
0,00,INDIA,20-24,11.518442
1,01,JAMMU & KASHMIR,20-24,29.580167
2,02,HIMACHAL PRADESH,20-24,8.206521
3,03,PUNJAB,15-19,45.763949
4,04,CHANDIGARH,50-69,42.774903
5,05,UTTARAKHAND,20-24,3.094835
6,06,HARYANA,15-19,7.318620
7,07,NCT OF DELHI,70+,14.999231
8,08,RAJASTHAN,15-19,2.884065
9,09,UTTAR PRADESH,20-24,2.443396


### Exporting the dataframe to 'age-india.csv'

In [ ]:
age_india.to_csv('OUTPUT/age-india.csv',index=False)